In [ ]:
# Copyright : Ryoungseob Kwon
# Cite this : Enabling 30 m maize mapping without ground reference leveraging multiple satellite platforms in Brazil and Argentina

import ee
ee.Authenticate()
ee.Initialize()

!pip install geemap==0.20.4 -q

import os
import math
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
L8_year = [2022]

# Administrative districts
input_States_dir = '/content/drive/MyDrive/01_MAIZE/06_multivariate/SA_States'
input_States_files = ['ARG_States_all.csv', 'BRZ_States_all.csv']

ARG_state_id = []
BRZ_state_id = []

for i in range(len(input_States_files)):
  target_States = pd.read_csv(os.path.join(input_States_dir, input_States_files[i]))
  for j in range(len(target_States)):
    if 'ARG' in input_States_files[i]:
      ARG_state_id.append(str(target_States.iloc[j, 0]))
    elif 'BRZ' in input_States_files[i]:
      BRZ_state_id.append(str(target_States.iloc[j, 0]))

# Random Forest
classifier = ee.Classifier.smileRandomForest(40)

In [ ]:
def revise_Bandnames_max(i):
  namei = ee.String(i)
  namei0 = ee.String('max_').cat(namei)
  return namei0

def revise_Bandnames_min(i):
  namei = ee.String(i)
  namei0 = ee.String('min_').cat(namei)
  return namei0

In [ ]:
def addDependents(image):
  years = image.date().difference('1970-01-01', 'year')
  time_radians = ee.Image(years.multiply(2 * math.pi)).rename('t')
  constant = ee.Image(1)
  return image.addBands(constant).addBands(time_radians.float())

def addHarmonics(freqs):
  def inner_function(image):
    frequencies = ee.Image.constant(freqs)
    time = ee.Image(image).select('t')
    cosines = time.multiply(frequencies).cos().rename(cosNames)
    sines = time.multiply(frequencies).sin().rename(sinNames)
    return image.addBands(cosines).addBands(sines)
  return inner_function

harmonics = 2
harmonicFrequencies = ee.List.sequence(1, harmonics)

cosNames = ee.List(["cos_1", "cos_2"])
sinNames = ee.List(["sin_1", "sin_2"])
independents = ee.List(['constant', 't']).cat(cosNames).cat(sinNames)

def prepSrL8(image):
  qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturation_mask = image.select('QA_RADSAT').eq(0)
  def get_factor_img(factor_names):
    factor_list = image.toDictionary().select(factor_names).values()
    return ee.Image.constant(factor_list)
  scale_img = get_factor_img(['REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'])
  offset_img = get_factor_img(['REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'])
  scaled = image.select('SR_B.|ST_B10').multiply(scale_img).add(offset_img)
  return image.addBands(scaled, None, True).updateMask(qa_mask).updateMask(saturation_mask)

def vegetation_index_L8(image):
  blueBand  = 'SR_B2'
  greenBand = 'SR_B3'
  redBand   = 'SR_B4'
  nirBand   = 'SR_B5'
  swir1Band = 'SR_B6'
  swir2Band = 'SR_B7'

  ndvi  = image.expression('(NIR - RED) / (NIR + RED)', {'NIR': image.select(nirBand), 'RED': image.select(redBand)}).rename('NDVI').copyProperties(image, ["system:time_start"])
  lswi  = image.expression('(NIR - SWIR1) / (NIR + SWIR1)', {'NIR': image.select(nirBand), 'SWIR1': image.select(swir1Band)}).rename('LSWI').copyProperties(image, ["system:time_start"])
  evi   = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {'NIR': image.select(nirBand), 'RED': image.select(redBand), 'BLUE': image.select(blueBand)}).rename('EVI').copyProperties(image, ["system:time_start"])
  gcvi  = image.expression('NIR / Green - 1', {'NIR': image.select(nirBand), 'Green': image.select(greenBand)}).rename('GCVI').copyProperties(image, ["system:time_start"])
  ndti  = image.expression('(SWIR1 - SWIR2) / (SWIR1 + SWIR2)', {'SWIR1': image.select(swir1Band), 'SWIR2': image.select(swir2Band)}).rename('NDTI').copyProperties(image, ["system:time_start"])
  bsi   = image.expression('((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))', {'SWIR1': image.select(swir1Band), 'RED': image.select(redBand), 'NIR': image.select(nirBand), 'BLUE': image.select(blueBand)}).rename('BSI').copyProperties(image, ["system:time_start"])

  new_img = image.addBands(ndvi).addBands(lswi).addBands(evi).addBands(gcvi).addBands(ndti).addBands(bsi)
  return new_img

def maize_classification_L8(ls8_crop_season, ls8_to_classify):
  ls8_crop_season_cloudless = ls8_crop_season.map(prepSrL8)
  ls8_to_classify_cloudless = ls8_to_classify.map(prepSrL8)

  ls8_crop_season_vi = ls8_crop_season_cloudless.map(vegetation_index_L8).select(['NDVI', 'LSWI', 'EVI', 'GCVI', 'NDTI'])
  ls8_to_classify_vi = ls8_to_classify_cloudless.map(vegetation_index_L8).select(['NDVI', 'LSWI', 'EVI', 'GCVI', 'NDTI'])

  ls8_crop_season_max = ls8_crop_season_vi.max()
  ls8_to_classify_max = ls8_to_classify_vi.max()

  ls8_crop_season_bsi = ls8_crop_season_cloudless.map(vegetation_index_L8).select(['BSI'])
  ls8_to_classify_bsi = ls8_to_classify_cloudless.map(vegetation_index_L8).select(['BSI'])

  ls8_crop_season_min = ls8_crop_season_bsi.min()
  ls8_to_classify_min = ls8_to_classify_bsi.min()

  bandname_max     = ls8_crop_season_max.bandNames().map(revise_Bandnames_max)
  bandname_min     = ls8_crop_season_min.bandNames().map(revise_Bandnames_min)

  ls8_crop_season_max     = ls8_crop_season_max.rename(bandname_max)
  ls8_crop_season_min     = ls8_crop_season_min.rename(bandname_min)

  ls8_to_classify_max     = ls8_to_classify_max.rename(bandname_max)
  ls8_to_classify_min     = ls8_to_classify_min.rename(bandname_min)

  harmonic_train    = ls8_crop_season_cloudless.map(vegetation_index_L8).map(addDependents).map(addHarmonics(harmonicFrequencies))
  harmonic_classify = ls8_to_classify_cloudless.map(vegetation_index_L8).map(addDependents).map(addHarmonics(harmonicFrequencies))

  # The dependent variable we are modeling.
  dependent1 = 'NDVI'
  dependent2 = 'LSWI'
  dependent3 = 'EVI'
  dependent4 = 'GCVI'
  dependent6 = 'NDTI'
  dependent7 = 'BSI'

  # NDVI Coefficients
  harmonicTrend_train1 = harmonic_train.select(independents.add(dependent1)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  harmonicTrend_classify1 = harmonic_classify.select(independents.add(dependent1)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  coefficients_train1 = harmonicTrend_train1.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  coefficients_classify1 = harmonicTrend_classify1.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  sin_train1 = coefficients_train1.select('sin_2').rename('sin_NDVI')
  cos_train1 = coefficients_train1.select('cos_2').rename('cos_NDVI')
  sin_classify1 = coefficients_classify1.select('sin_2').rename('sin_NDVI')
  cos_classify1 = coefficients_classify1.select('cos_2').rename('cos_NDVI')

  # LSWI Coefficients
  harmonicTrend_train2 = harmonic_train.select(independents.add(dependent2)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  harmonicTrend_classify2 = harmonic_classify.select(independents.add(dependent2)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  coefficients_train2 = harmonicTrend_train2.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  coefficients_classify2 = harmonicTrend_classify2.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  sin_train2 = coefficients_train2.select('sin_2').rename('sin_LSWI')
  cos_train2 = coefficients_train2.select('cos_2').rename('cos_LSWI')
  sin_classify2 = coefficients_classify2.select('sin_2').rename('sin_LSWI')
  cos_classify2 = coefficients_classify2.select('cos_2').rename('cos_LSWI')

  # EVI Coefficients
  harmonicTrend_train3 = harmonic_train.select(independents.add(dependent3)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  harmonicTrend_classify3 = harmonic_classify.select(independents.add(dependent3)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  coefficients_train3 = harmonicTrend_train3.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  coefficients_classify3 = harmonicTrend_classify3.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  sin_train3 = coefficients_train3.select('sin_2').rename('sin_EVI')
  cos_train3 = coefficients_train3.select('cos_2').rename('cos_EVI')
  sin_classify3 = coefficients_classify3.select('sin_2').rename('sin_EVI')
  cos_classify3 = coefficients_classify3.select('cos_2').rename('cos_EVI')

  # GCVI Coefficients
  harmonicTrend_train4 = harmonic_train.select(independents.add(dependent4)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  harmonicTrend_classify4 = harmonic_classify.select(independents.add(dependent4)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  coefficients_train4 = harmonicTrend_train4.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  coefficients_classify4 = harmonicTrend_classify4.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  sin_train4 = coefficients_train4.select('sin_2').rename('sin_GCVI')
  cos_train4 = coefficients_train4.select('cos_2').rename('cos_GCVI')
  sin_classify4 = coefficients_classify4.select('sin_2').rename('sin_GCVI')
  cos_classify4 = coefficients_classify4.select('cos_2').rename('cos_GCVI')

  # NDTI Coefficients
  harmonicTrend_train5 = harmonic_train.select(independents.add(dependent6)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  harmonicTrend_classify5 = harmonic_classify.select(independents.add(dependent6)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  coefficients_train5 = harmonicTrend_train5.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  coefficients_classify5 = harmonicTrend_classify5.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  sin_train5 = coefficients_train5.select('sin_2').rename('sin_NDTI')
  cos_train5 = coefficients_train5.select('cos_2').rename('cos_NDTI')
  sin_classify5 = coefficients_classify5.select('sin_2').rename('sin_NDTI')
  cos_classify5 = coefficients_classify5.select('cos_2').rename('cos_NDTI')

  # BSI Coefficients
  harmonicTrend_train6 = harmonic_train.select(independents.add(dependent7)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  harmonicTrend_classify6 = harmonic_classify.select(independents.add(dependent7)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
  coefficients_train6 = harmonicTrend_train6.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  coefficients_classify6 = harmonicTrend_classify6.select('coefficients').arrayProject([0]).arrayFlatten([independents])
  sin_train6 = coefficients_train6.select('sin_2').rename('sin_BSI')
  cos_train6 = coefficients_train6.select('cos_2').rename('cos_BSI')
  sin_classify6 = coefficients_classify6.select('sin_2').rename('sin_BSI')
  cos_classify6 = coefficients_classify6.select('cos_2').rename('cos_BSI')

  train_bands = ls8_crop_season_max.addBands(ls8_crop_season_min).addBands(sin_train1).addBands(cos_train1).addBands(sin_train2).addBands(cos_train2).addBands(sin_train3).addBands(cos_train3).addBands(sin_train4).addBands(cos_train4).addBands(sin_train5).addBands(cos_train5).addBands(sin_train6).addBands(cos_train6)
  test_bands  = ls8_to_classify_max.addBands(ls8_to_classify_min).addBands(sin_classify1).addBands(cos_classify1).addBands(sin_classify2).addBands(cos_classify2).addBands(sin_classify3).addBands(cos_classify3).addBands(sin_classify4).addBands(cos_classify4).addBands(sin_classify5).addBands(cos_classify5).addBands(sin_classify6).addBands(cos_classify6)

  return train_bands, test_bands

def process_output(classified_img):
  image1 = ee.Image.constant(1)
  output_img = classified_img.add(image1)
  output_img_Int = output_img.toInt16()
  maize_result_mask = output_img_Int.neq(0)
  output_maize = output_img_Int.updateMask(maize_result_mask)
  return output_maize

In [ ]:
def maize_mapping_L8(target_year_to_classify, resolution, country):

  state_datasets = ee.FeatureCollection('FAO/GAUL/2015/level1')
  target_date_former = ee.Date(str(target_year_to_classify) + '-09-01')
  target_date_latter = ee.Date(str(target_year_to_classify + 1) + '-09-01')

  if country == 'ARG':
    fname_former = 'arg_'
    target_folder = 'arg_maize_maps'
    target_bounds = ee.FeatureCollection('users/twinsben94/argentina_bounds')
    target_states_id = ARG_state_id
    maize_loc = ee.FeatureCollection("users/twinsben94/arg_maize")         # 'maize' = 0
    others_loc = ee.FeatureCollection("users/twinsben94/arg_others")       # 'others' = 1
    all_loc = maize_loc.merge(others_loc)

  elif country == 'BRZ':
    fname_former = 'brz_'
    target_folder = 'brz_maize_maps'
    target_bounds = ee.FeatureCollection('users/twinsben94/brazil_bounds')
    target_states_id = BRZ_state_id
    maize_loc = ee.FeatureCollection("users/twinsben94/brz_maize")         # '1st maize' = 0, '2nd maize' = 1
    others_loc = ee.FeatureCollection("users/twinsben94/brz_others")       # 'others' = 2
    all_loc = maize_loc.merge(others_loc)

  ls8_crop_season = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2020-09-01', '2021-09-01').filterBounds(all_loc)
  ls8_to_classify = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(target_date_former, target_date_latter).filterBounds(target_bounds)

  train_bands, test_bands = maize_classification_L8(ls8_crop_season, ls8_to_classify)
  train_points = train_bands.sampleRegions(collection = all_loc, properties = ['maize'], scale = 30, tileScale = 16)
  trained_classifier = classifier.train(features = train_points, classProperty = 'maize', inputProperties = train_bands.bandNames())

  for num in target_states_id:
    fname = fname_former + str(target_year_to_classify) + '_' + str(num)
    target_state_ee = state_datasets.filterMetadata('ADM1_CODE', 'equals', int(num))
    export = ee.FeatureCollection.geometry(target_state_ee)
    image_to_classify = test_bands.clip(export)
    classified_images = image_to_classify.classify(trained_classifier)
    output_img = process_output(classified_images)

    task = ee.batch.Export.image.toDrive(image = output_img, fileFormat = 'GeoTIFF', folder = target_folder, description = fname, region = export, scale = int(resolution), crs = "EPSG:4326", maxPixels = 1e13)
    task.start()

In [ ]:
# Argentina
for year in L8_year:
  maize_mapping_L8(target_year_to_classify = year, resolution = 30, country = 'ARG')

# Brazil
for year in L8_year:
  maize_mapping_L8(target_year_to_classify = year, resolution = 30, country = 'BRZ')
